# Hyperparameters

In [1]:
CFG = 'cfgs/brain/brain_pospoolxyz.yaml'#'cfgs/brain/brain_pointwisemlp.yaml'
IS_CONF = True
FINE_TUNE = True
IS_EXPERIMENT = True
BRAINS = range(26)
DEVICE = 1

#For early stopping
EXP_NAME = '4'
PATIENCE = 150

# For fine-tuning
PRETRAINED_MODEL_PATH = 'checkpoints/3.pth'

#For loss
LOSS_TYPE = 'BCE'
IS_KUNI = False
KUNI_AGG = 'mean'
KUNI_LAM = 1
IS_SEP_LOSS = False

#lala
RES_PATH = 'Results/'+EXP_NAME
IS_CROP = True

# Importing libraries

In [2]:
if IS_EXPERIMENT:
    from comet_ml import Experiment

    experiment = Experiment(
        api_key="1cDG73F9830XhuYfWn4JJ2JEH",
        project_name="project-dl-bia",
        workspace="rukubrakov",
    )

COMET INFO: Experiment is live on comet.ml https://www.comet.ml/rukubrakov/project-dl-bia/a723175060a44435991567021f4c9a1d



In [3]:
from utils.fcd import *
from utils.crop import get_loader_crop
warnings.filterwarnings("ignore")

In [4]:
device = torch.device('cuda') if torch.cuda.is_available() else 'cpu'
torch.cuda.set_device(DEVICE)
torch.cuda.current_device()

1

In [5]:
config = config_seting(CFG)

# Data processing, train and validation

In [ ]:
files_done = [int(x.split('_')[-1]) for x in glob.glob(RES_PATH+'_*')]

for e in tqdm(BRAINS):
    if e in files_done:
        continue
    train_loader, test_loader = get_loader_crop(batch_size = config.batch_size,num_points = config.num_points, 
                                       config = config,test_brain = e )
    
    model, criterion = build_multi_part_segmentation(config = config,
                                                     weights = None,
                                                     type = LOSS_TYPE,
                                                     is_kuni = IS_KUNI,
                                                     kuni_agg = KUNI_AGG,
                                                     kuni_lam = KUNI_LAM,
                                                     is_sep_loss = IS_SEP_LOSS
                                                    )
    if FINE_TUNE:
        model.load_state_dict(torch.load(PRETRAINED_MODEL_PATH))
    model.cuda()
    criterion.cuda()
    
    n_data = len(train_loader.dataset)
    print(f"length of training dataset: {n_data}")
    n_data = len(test_loader.dataset)
    print(f"length of testing dataset: {n_data}")
    
   
    if config.optimizer == 'sgd':
        optimizer = torch.optim.SGD(model.parameters(),
                                    lr=config.batch_size * dist.get_world_size() / 16 * config.base_learning_rate,
                                    momentum=config.momentum,
                                    weight_decay=config.weight_decay)
    elif config.optimizer == 'adam':
        optimizer = torch.optim.Adam(model.parameters(),
                                     lr=config.base_learning_rate,
                                     weight_decay=config.weight_decay)
    elif config.optimizer == 'adamW':
        optimizer = torch.optim.AdamW(model.parameters(),
                                      lr=config.base_learning_rate,
                                      weight_decay=config.weight_decay)
    else:
        raise NotImplementedError(f"Optimizer {config.optimizer} not supported")
    scheduler = get_scheduler(optimizer, len(train_loader), config)
    
    test_iou = []
    test_losses = []
    test_dice = []
    test_roc = []
    test_tpr = []
    test_fpr = []
    epochs = []
    minimal_loss = 1e8
    early_stopping = EarlyStopping(patience=PATIENCE, verbose=True, 
                                   path = f'../pytorch/checkpoints/{EXP_NAME}_{e}.pth')
    
    for epoch in tqdm(range(config.start_epoch, config.epochs + 1)):

        tic = time.time()
        loss, opt, roc = train(epoch, train_loader, model, criterion, optimizer, scheduler,
                               config,is_kuni = IS_KUNI,is_sep_loss = IS_SEP_LOSS)
        if IS_EXPERIMENT:
            experiment.log_metric('roc_train',roc, epoch = epoch)
            experiment.log_metric('lr',optimizer.param_groups[0]['lr'], epoch = epoch)
        
        tmp = validate(epoch, test_loader, model, criterion, config, num_votes=1, 
                       is_conf = IS_CONF,is_kuni = IS_KUNI,is_sep_loss = IS_SEP_LOSS, is_crop = IS_CROP)
        if IS_CONF:
            loss_test,acc, msIoU, mIoU,confs, roc,opt, tpr,fpr,dice_score = tmp
        else:
            loss_test,acc, msIoU, mIoU, roc,opt,tpr,fpr,dice_score = tmp
            
        
        if IS_EXPERIMENT:   
            experiment.log_metric('brain',e, epoch = epoch)
            experiment.log_metric('optimal_cutoff_test',opt, epoch = epoch)
            experiment.log_metric('tpr_test',tpr, epoch = epoch)
            experiment.log_metric('fpr_test',fpr, epoch = epoch)
            experiment.log_metric('roc_test',roc, epoch = epoch)
            experiment.log_metric('accuracy_test',acc, epoch = epoch)
            if IS_CROP:
                experiment.log_metric('dice_score_test',dice_score[0], epoch = epoch)
                experiment.log_metric('IoU_test',msIoU[0], epoch = epoch)
                experiment.log_metric('dice_score_proba_test',dice_score[1], epoch = epoch)
                experiment.log_metric('IoU_proba_test',msIoU[1], epoch = epoch)
                experiment.log_metric('dice_score_ideal_test',dice_score[2], epoch = epoch)
                experiment.log_metric('IoU_ideal_test',msIoU[2], epoch = epoch)
                experiment.log_metric('FCD_Contrast_test',msIoU[3], epoch = epoch)
            else:
                experiment.log_metric('dice_score_test',dice_score, epoch = epoch)
                experiment.log_metric('IoU_test',msIoU, epoch = epoch)
            if IS_SEP_LOSS:
                experiment.log_metric('loss_train',loss[0]+loss[1], epoch = epoch)
                experiment.log_metric('loss_train_base',loss[0], epoch = epoch)
                experiment.log_metric('loss_train_kuni',loss[1], epoch = epoch)
                experiment.log_metric('loss_test',loss_test[0]+loss_test[1], epoch = epoch)
                experiment.log_metric('loss_test_base',loss_test[0], epoch = epoch)
                experiment.log_metric('loss_test_kuni',loss_test[1], epoch = epoch)
            else:
                experiment.log_metric('loss_train',loss, epoch = epoch)
                experiment.log_metric('loss_test',loss_test, epoch = epoch)
            
        if IS_EXPERIMENT:
            if IS_CONF:
                experiment.log_confusion_matrix(title=f"Test confusion epoch = {epoch}", matrix = confs, labels = ['No FCD', 'FCD'])
        if IS_SEP_LOSS:   
            test_losses.append(loss_test[0])
        else:
            test_losses.append(loss_test)
        test_iou.append(msIoU)
        test_roc.append(roc)
        test_dice.append(dice_score)
        test_tpr.append(tpr)
        test_fpr.append(fpr)
        epochs.append(epoch)

        early_stopping(-roc, model)
        
        if early_stopping.early_stop:
            print("Early stopping")
            break
        df = pd.DataFrame({'epoch':epochs,'iou':test_iou,'loss':test_losses,'dice_score':test_dice,
                       'roc':test_roc,'tpr':test_tpr,'fpr':test_fpr})
        df.to_csv(RES_PATH+'_'+str(e),index = False)
    del model
    

  0%|          | 0/500 [00:00<?, ?it/s]

length of training dataset: 25
length of testing dataset: 115
Validation loss decreased (inf --> -0.439883).  Saving model ...



  0%|          | 2/500 [00:21<1:30:41, 10.93s/it]

Validation loss decreased (-0.439883 --> -0.509566).  Saving model ...
Validation loss decreased (-0.509566 --> -0.511593).  Saving model ...



  1%|          | 4/500 [00:43<1:29:34, 10.84s/it]

...

EarlyStopping counter: 24 out of 150



 46%|████▌     | 230/500 [41:29<48:42, 10.83s/it]

EarlyStopping counter: 25 out of 150



 46%|████▌     | 231/500 [41:41<48:32, 10.83s/it]

EarlyStopping counter: 26 out of 150


In [3]:
! nvidia-smi

Wed May 26 07:25:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64       Driver Version: 440.64       CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  GeForce GTX 108...  Off  | 00000000:01:00.0 Off |                  N/A |
| 20%   34C    P8    16W / 250W |    765MiB / 11177MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  Off  | 00000000:02:00.0 Off |                  N/A |
| 38%   71C    P2   187W / 250W |  10515MiB / 11178MiB |    100%      Default |
+-------------------------------+----------------------+----------------------+
        